In [ ]:
import os
import numpy as np
import cv2
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import SwinConfig, SwinModel

IMG_SIZE = 256
BATCH_SIZE = 16
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {DEVICE}")

# Thay đổi đường dẫn thư mục tùy theo máy của bạn
train_img_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\train\image'
train_mask_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\train\label'
val_img_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\val\image'
val_mask_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\val\label'

# --- Thu thập đường dẫn tệp ảnh và mask ---
train_img_paths = sorted([os.path.join(train_img_dir, f) for f in os.listdir(train_img_dir)])
train_mask_paths = sorted([os.path.join(train_mask_dir, f) for f in os.listdir(train_mask_dir)])
val_img_paths = sorted([os.path.join(val_img_dir, f) for f in os.listdir(val_img_dir)])
val_mask_paths = sorted([os.path.join(val_mask_dir, f) for f in os.listdir(val_mask_dir)])

# --- Lớp Dataset tùy chỉnh ---
class CrackDetectionDataset(Dataset):
    def __init__(self, image_filenames, mask_filenames, augment=False):
        self.image_filenames = image_filenames
        self.mask_filenames = mask_filenames
        self.augment = augment
        
        if len(self.image_filenames) != len(self.mask_filenames):
            raise ValueError("Số lượng tệp ảnh và tệp mask không khớp.")

    def __len__(self):
        return len(self.image_filenames)

    def pad_and_crop_image_and_mask(self, img, mask, target_size=IMG_SIZE, window_size=4):
        h, w = img.shape[:2]
        new_h = max(target_size, ((h + window_size - 1) // window_size) * window_size)
        new_w = max(target_size, ((w + window_size - 1) // window_size) * window_size)

        padded_img = np.zeros((new_h, new_w, 3), dtype=img.dtype)
        padded_mask = np.zeros((new_h, new_w), dtype=mask.dtype)

        padded_img[:h, :w, :] = img
        padded_mask[:h, :w] = mask

        cropped_img = padded_img[:target_size, :target_size, :]
        cropped_mask = padded_mask[:target_size, :target_size]

        return cropped_img, cropped_mask

    def __getitem__(self, idx):
        img = cv2.imread(self.image_filenames[idx])
        if img is None:
            raise ValueError(f"Không thể đọc tệp ảnh: {self.image_filenames[idx]}")

        mask = cv2.imread(self.mask_filenames[idx], cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise ValueError(f"Không thể đọc tệp mask: {self.mask_filenames[idx]}")

        img, mask = self.pad_and_crop_image_and_mask(img, mask, target_size=IMG_SIZE, window_size=4)

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        img = img.astype(np.float32) / 255.0
        mask = mask.astype(np.float32) / 255.0
        
        if self.augment:
            if random.random() < 0.5:
                img = cv2.flip(img, 1)
                mask = cv2.flip(mask, 1)
            if random.random() < 0.5:
                img = cv2.flip(img, 0)
                mask = cv2.flip(mask, 0)
        
        if mask.ndim == 2:
            mask = np.expand_dims(mask, axis=-1)

        if mask.shape != (IMG_SIZE, IMG_SIZE, 1):
            raise ValueError(f"Hình dạng mask không mong muốn {mask.shape} tại chỉ số {idx} sau khi xử lý.")

        img_tensor = torch.from_numpy(img).permute(2, 0, 1)
        mask_tensor = torch.from_numpy(mask).permute(2, 0, 1)

        return img_tensor, mask_tensor

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super().__init__()
        self.upsample = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        
        if skip_channels > 0:
            self.conv_block = ConvBlock(out_channels + skip_channels, out_channels)
        else:
            self.conv_block = ConvBlock(out_channels, out_channels)

    def forward(self, x, skip_features=None):
        x = self.upsample(x)
        
        if skip_features is not None and skip_features.size(1) > 0:
            if x.shape[2:] != skip_features.shape[2:]:
                diffY = skip_features.size()[2] - x.size()[2]
                diffX = skip_features.size()[3] - x.size()[3]
                x = nn.functional.pad(x, [diffX // 2, diffX - diffX // 2,
                                          diffY // 2, diffY - diffY // 2])
            x = torch.cat([x, skip_features], dim=1)
        
        x = self.conv_block(x)
        return x

class SwinUNet(nn.Module):
    def __init__(self, input_channels=3, num_classes=1):
        super().__init__()
        self.IMG_SIZE = IMG_SIZE

        config = SwinConfig(image_size=self.IMG_SIZE, num_channels=input_channels, 
                            patch_size=4, embed_dim=96, depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24],
                            window_size=7, mlp_ratio=4., qkv_bias=True, hidden_dropout_prob=0.0, 
                            attention_probs_dropout_prob=0.0, drop_path_rate=0.1, 
                            hidden_act="gelu", use_absolute_embeddings=False, 
                            patch_norm=True, initializer_range=0.02, layer_norm_eps=1e-05,
                            out_features=["stage1", "stage2", "stage3", "stage4"])
        self.swin = SwinModel(config)

        self.bottleneck = ConvBlock(config.embed_dim * 8, config.embed_dim * 8)

        self.decoder4 = DecoderBlock(in_channels=config.embed_dim * 8, skip_channels=config.embed_dim * 4, out_channels=config.embed_dim * 4)
        self.decoder3 = DecoderBlock(in_channels=config.embed_dim * 4, skip_channels=config.embed_dim * 2, out_channels=config.embed_dim * 2)
        self.decoder2 = DecoderBlock(in_channels=config.embed_dim * 2, skip_channels=config.embed_dim * 1, out_channels=config.embed_dim * 1)
        
        self.decoder1 = DecoderBlock(in_channels=config.embed_dim * 1, skip_channels=0, out_channels=config.embed_dim // 2)

        self.final_upsample = DecoderBlock(in_channels=config.embed_dim // 2, skip_channels=0, out_channels=config.embed_dim // 4)

        self.final_conv = nn.Conv2d(config.embed_dim // 4, num_classes, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        outputs = self.swin(pixel_values=x, output_hidden_states=True)
        # print("\nHidden States từ SwinModel:") # Có thể bỏ comment nếu cần debug
        # for i, hs in enumerate(outputs.hidden_states):
        #     print(f"  Hidden state {i} shape: {hs.shape}")

        encoder_features = []

        # outputs.hidden_states[0] là Patch Embedding (kích thước 64x64, 96 kênh)
        hs0 = outputs.hidden_states[0]
        batch_size, num_patches, embed_dim = hs0.shape
        side = int(np.sqrt(num_patches))
        encoder_features.append(hs0.permute(0, 2, 1).reshape(batch_size, embed_dim, side, side))

        # outputs.hidden_states[1] là Stage 1 (kích thước 32x32, 192 kênh)
        hs1 = outputs.hidden_states[1]
        batch_size, num_patches, embed_dim = hs1.shape
        side = int(np.sqrt(num_patches))
        encoder_features.append(hs1.permute(0, 2, 1).reshape(batch_size, embed_dim, side, side))

        # outputs.hidden_states[2] là Stage 2 (kích thước 16x16, 384 kênh)
        hs2 = outputs.hidden_states[2]
        batch_size, num_patches, embed_dim = hs2.shape
        side = int(np.sqrt(num_patches))
        encoder_features.append(hs2.permute(0, 2, 1).reshape(batch_size, embed_dim, side, side))

        # outputs.hidden_states[4] là đầu ra của Stage 4, dùng cho bottleneck
        x_bottleneck = outputs.hidden_states[4]
        batch_size, num_patches, embed_dim = x_bottleneck.shape
        side = int(np.sqrt(num_patches))
        x_bottleneck = x_bottleneck.permute(0, 2, 1).reshape(batch_size, embed_dim, side, side)
        
        x = self.bottleneck(x_bottleneck)

        x = self.decoder4(x, encoder_features[2])
        x = self.decoder3(x, encoder_features[1])
        x = self.decoder2(x, encoder_features[0])
        x = self.decoder1(x)
        x = self.final_upsample(x)

        outputs = self.final_conv(x)
        outputs = self.sigmoid(outputs)

        return outputs

# --- Các hàm tính toán chỉ số mới (IoU và F1-Score) ---
def calculate_metrics(predicted_masks, true_masks, smooth=1e-6):
    # predicted_masks và true_masks phải là tensor nhị phân (0 hoặc 1)
    
    intersection = (predicted_masks * true_masks).sum()
    union = (predicted_masks + true_masks).sum() - intersection
    
    # IoU
    iou = (intersection + smooth) / (union + smooth)
    
    # F1-Score (Dice Coefficient)
    # F1 = 2 * (precision * recall) / (precision + recall)
    # Precision = TP / (TP + FP)
    # Recall = TP / (TP + FN)
    
    # TP (True Positives): pixel dự đoán là 1, thực tế là 1
    # FP (False Positives): pixel dự đoán là 1, thực tế là 0
    # FN (False Negatives): pixel dự đoán là 0, thực tế là 1
    
    # TP = intersection
    # FP = (predicted_masks == 1).sum() - TP
    # FN = (true_masks == 1).sum() - TP

    dice = (2. * intersection + smooth) / ((predicted_masks.sum() + true_masks.sum()) + smooth)
    f1_score = dice # F1-score và Dice Coefficient là như nhau cho phân loại nhị phân

    return iou.item(), f1_score.item()


# Thêm tham số start_epoch và best_val_loss_so_far để tiếp tục từ checkpoint
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, callbacks_config, start_epoch=0, best_val_loss_so_far=float('inf')):
    best_val_loss = best_val_loss_so_far # Khởi tạo best_val_loss từ giá trị đã tải
    patience_counter = 0
    model_checkpoint_path = callbacks_config.get('checkpoint_path', 'swin_unet_best_pytorch.pth')

    # Vòng lặp epoch bắt đầu từ start_epoch
    for epoch in range(start_epoch, num_epochs):
        model.train()
        running_loss = 0.0
        running_iou = 0.0
        running_f1 = 0.0
        print(f"Epoch {epoch+1}/{num_epochs} Bắt đầu...")
        for batch_idx, (images, masks) in enumerate(train_loader):
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, masks)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

            # Tính toán IoU và F1-Score
            predicted_masks = (outputs > 0.5).float()
            
            batch_iou, batch_f1 = calculate_metrics(predicted_masks, masks)
            running_iou += batch_iou * images.size(0)
            running_f1 += batch_f1 * images.size(0)

            if (batch_idx + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_iou = running_iou / len(train_loader.dataset)
        epoch_f1 = running_f1 / len(train_loader.dataset)
        # Thay đổi dòng in để hiển thị IoU và F1
        print(f"Epoch {epoch+1} Kết thúc - Mất mát Huấn luyện: {epoch_loss:.4f}, IoU Huấn luyện: {epoch_iou:.4f}, F1-Score Huấn luyện: {epoch_f1:.4f}")

        model.eval()
        val_loss = 0.0
        val_iou = 0.0
        val_f1 = 0.0

        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(DEVICE)
                masks = masks.to(DEVICE)

                outputs = model(images)
                loss = criterion(outputs, masks)

                val_loss += loss.item() * images.size(0)

                # Tính toán IoU và F1-Score cho tập xác thực
                predicted_masks = (outputs > 0.5).float()
                
                batch_iou, batch_f1 = calculate_metrics(predicted_masks, masks)
                val_iou += batch_iou * images.size(0)
                val_f1 += batch_f1 * images.size(0)

        val_loss /= len(val_loader.dataset)
        val_iou /= len(val_loader.dataset)
        val_f1 /= len(val_loader.dataset)
        # Thay đổi dòng in để hiển thị IoU và F1
        print(f"Mất mát Xác thực: {val_loss:.4f}, IoU Xác thực: {val_iou:.4f}, F1-Score Xác thực: {val_f1:.4f}")

        # --- Callbacks: Early Stopping và Model Checkpoint ---
        # Vẫn sử dụng val_loss để quyết định lưu mô hình tốt nhất
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            print(f"Mất mát xác thực tốt nhất được cập nhật: {best_val_loss:.4f}. Lưu mô hình và trạng thái...")
            # Lưu checkpoint đầy đủ
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_loss': best_val_loss,
            }, model_checkpoint_path)
        else:
            patience_counter += 1
            print(f"Mất mát xác thực không cải thiện. Sự kiên nhẫn: {patience_counter}/{callbacks_config['patience']}")
            if patience_counter >= callbacks_config['patience']:
                print("Dừng sớm!")
                break

# --- Khởi tạo và tải dữ liệu ---
train_dataset = CrackDetectionDataset(train_img_paths, train_mask_paths, augment=True)
val_dataset = CrackDetectionDataset(val_img_paths, val_mask_paths, augment=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# --- Khởi tạo mô hình, optimizer, criterion ---
model = SwinUNet(input_channels=3, num_classes=1).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

print(model)

callbacks_config = {
    'patience': 10,
    'checkpoint_path': 'swin_unet_best_pytorch.pth'
}

# --- Logic để tiếp tục huấn luyện từ checkpoint ---
start_epoch = 0
best_val_loss_so_far = float('inf')
checkpoint_path = callbacks_config['checkpoint_path']

if os.path.exists(checkpoint_path):
    print(f"Phát hiện checkpoint tại {checkpoint_path}. Đang tải để tiếp tục huấn luyện...")
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1 # Bắt đầu từ epoch tiếp theo sau khi checkpoint được lưu
    best_val_loss_so_far = checkpoint['best_val_loss']
    
    print(f"Đã tải checkpoint từ Epoch {start_epoch-1}. Tiếp tục huấn luyện từ Epoch {start_epoch}.")
    print(f"Mất mát xác thực tốt nhất trước đó: {best_val_loss_so_far:.4f}")
else:
    print("Không tìm thấy checkpoint. Bắt đầu huấn luyện từ đầu (Epoch 0).")

print("\nBắt đầu huấn luyện mô hình Swin-Unet...")
# Gọi hàm huấn luyện với các tham số mới
train_model(model, train_loader, val_loader, optimizer, criterion, 
            num_epochs=100, callbacks_config=callbacks_config,
            start_epoch=start_epoch, best_val_loss_so_far=best_val_loss_so_far)

print("\nQuá trình huấn luyện đã hoàn tất.")

c:\Users\Admin\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Sử dụng thiết bị: cuda
SwinUNet(
  (swin): SwinModel(
    (embeddings): SwinEmbeddings(
      (patch_embeddings): SwinPatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SwinEncoder(
      (layers): ModuleList(
        (0): SwinStage(
          (blocks): ModuleList(
            (0): SwinLayer(
              (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attention): SwinAttention(
                (self): SwinSelfAttention(
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96, bias=True)
                  (value): Linear(in_features=96, out_features=96, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
                (output): SwinSelfOutput(
  

In [6]:
import os
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import SwinConfig, SwinModel

import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- Cấu hình ---
IMG_SIZE = 256
BATCH_SIZE = 8
SEED = 42

# --- Đặt Seed để tái lập kết quả ---
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {DEVICE}")

# Thay đổi đường dẫn thư mục tùy theo máy của bạn
train_img_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\train\image'
train_mask_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\train\label'
val_img_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\val\image'
val_mask_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\val\label'

# --- Thu thập đường dẫn tệp ảnh và mask ---
train_img_paths = sorted([os.path.join(train_img_dir, f) for f in os.listdir(train_img_dir)])
train_mask_paths = sorted([os.path.join(train_mask_dir, f) for f in os.listdir(train_mask_dir)])
val_img_paths = sorted([os.path.join(val_img_dir, f) for f in os.listdir(val_img_dir)])
val_mask_paths = sorted([os.path.join(val_mask_dir, f) for f in os.listdir(val_mask_dir)])


## Biến Đổi Albumentations

# Pipeline tăng cường dữ liệu cho huấn luyện
# Chúng ta sẽ thay đổi kích thước ảnh đến một kích thước lớn hơn một chút, sau đó cắt ngẫu nhiên,
# và cuối cùng thay đổi kích thước về IMG_SIZE.
# Điều này mô phỏng việc "cắt các phần ngẫu nhiên của ảnh" và sau đó thay đổi kích thước chúng về kích thước mục tiêu.
train_transform = A.Compose([
    # Thay đổi kích thước ảnh đến một kích thước lớn hơn một chút trước khi cắt
    # để đảm bảo có đủ không gian cho việc cắt và sau đó thay đổi kích thước về IMG_SIZE.
    # Chúng ta chọn 1.25 * IMG_SIZE làm kích thước trung gian lớn hơn.
    A.LongestMaxSize(max_size=int(IMG_SIZE * 1.25), interpolation=cv2.INTER_AREA),
    A.PadIfNeeded(min_height=int(IMG_SIZE * 1.25), min_width=int(IMG_SIZE * 1.25), border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0),
    A.RandomCrop(height=IMG_SIZE, width=IMG_SIZE, p=1.0), # Luôn thực hiện cắt ngẫu nhiên về IMG_SIZE
    A.HorizontalFlip(p=0.5), # Lật ngang ngẫu nhiên
    A.VerticalFlip(p=0.5),   # Lật dọc ngẫu nhiên
    # Chuẩn hóa ảnh. Mean và Std của ImageNet rất phổ biến khi dùng với các mô hình pre-trained.
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(), # Chuyển ảnh và mask sang PyTorch tensors
])

# Pipeline biến đổi cho xác thực (chỉ resize và chuẩn hóa)
val_transform = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE, interpolation=cv2.INTER_AREA),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

c:\Users\Admin\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Sử dụng thiết bị: cuda


C:\Users\Admin\AppData\Local\Temp\ipykernel_17800\3361398528.py:54: UserWarning: Argument(s) 'value, mask_value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(min_height=int(IMG_SIZE * 1.25), min_width=int(IMG_SIZE * 1.25), border_mode=cv2.BORDER_CONSTANT, value=0, mask_value=0),


In [7]:
class CrackDetectionDataset(Dataset):
    def __init__(self, image_filenames, mask_filenames, transform=None):
        self.image_filenames = image_filenames
        self.mask_filenames = mask_filenames
        self.transform = transform

        if len(self.image_filenames) != len(self.mask_filenames):
            raise ValueError("Số lượng tệp ảnh và tệp mask không khớp.")

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img = cv2.imread(self.image_filenames[idx])
        if img is None:
            raise ValueError(f"Không thể đọc tệp ảnh: {self.image_filenames[idx]}")

        # Đọc mask dưới dạng ảnh xám
        mask = cv2.imread(self.mask_filenames[idx], cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise ValueError(f"Không thể đọc tệp mask: {self.mask_filenames[idx]}")

        # Chuyển đổi ảnh sang RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Albumentations cần mask có giá trị 0 hoặc 1 (hoặc các lớp khác)
        # và không cần ở dạng float hoặc có thêm chiều kênh ngay tại đây
        mask = (mask > 127).astype(np.float32) # Chuyển mask về 0.0 hoặc 1.0

        if self.transform:
            # Albumentations mong đợi ảnh (H, W, C) và mask (H, W)
            augmented = self.transform(image=img, mask=mask)
            img_tensor = augmented['image']
            mask_tensor = augmented['mask']
        else:
            # Đây là fallback nếu không có transform, nhưng nên dùng val_transform
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
            mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
            img_tensor = torch.from_numpy(img.astype(np.float32) / 255.0).permute(2, 0, 1)
            mask_tensor = torch.from_numpy(mask.astype(np.float32)).unsqueeze(0) # Thêm chiều kênh cho mask

        # Đảm bảo mask có chiều kênh (1, H, W) và loại float
        if mask_tensor.ndim == 2:
            mask_tensor = mask_tensor.unsqueeze(0) # Add channel dim for (H,W) -> (1,H,W)
        
        return img_tensor, mask_tensor

In [8]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super().__init__()
        # Sử dụng Interpolate để upsample thay vì ConvTranspose2d,
        # giúp tránh các vấn đề 'checkerboard artifacts' và linh hoạt hơn với các kích thước không chia hết.
        # ConvTranspose2d vẫn ổn, nhưng Interpolate thường cho kết quả mịn hơn trong segmentation.
        self.upsample = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=1) # Conv1x1 để điều chỉnh kênh sau upsample
        )
        
        if skip_channels > 0:
            self.conv_block = ConvBlock(out_channels + skip_channels, out_channels)
        else:
            self.conv_block = ConvBlock(out_channels, out_channels)

    def forward(self, x, skip_features=None):
        x = self.upsample(x)
        
        if skip_features is not None:
            # Đảm bảo kích thước không gian khớp trước khi concatenate
            if x.shape[2:] != skip_features.shape[2:]:
                x = nn.functional.interpolate(x, size=skip_features.shape[2:], mode='bilinear', align_corners=True)
            x = torch.cat([x, skip_features], dim=1)
        
        x = self.conv_block(x)
        return x

class SwinUNet(nn.Module):
    def __init__(self, input_channels=3, num_classes=1):
        super().__init__()
        self.IMG_SIZE = IMG_SIZE

        # Đảm bảo out_features bao gồm "patch_embeddings" để có thể dùng làm skip connection đầu tiên
        config = SwinConfig(image_size=self.IMG_SIZE, num_channels=input_channels, 
                            patch_size=4, embed_dim=96, depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24],
                            window_size=7, mlp_ratio=4., qkv_bias=True, hidden_dropout_prob=0.0, 
                            attention_probs_dropout_prob=0.0, drop_path_rate=0.1, 
                            hidden_act="gelu", use_absolute_embeddings=False, 
                            patch_norm=True, initializer_range=0.02, layer_norm_eps=1e-05,
                            out_features=["patch_embeddings", "stage1", "stage2", "stage3", "stage4"])
        self.swin = SwinModel(config)

        # Kích thước kênh đầu ra của các stage Swin (ví dụ cho embed_dim=96):
        # patch_embeddings: 96 (từ 256/4 = 64x64)
        # stage1: 192 (96*2) (từ 256/8 = 32x32)
        # stage2: 384 (96*4) (từ 256/16 = 16x16)
        # stage3: 768 (96*8) (từ 256/32 = 8x8)
        # stage4: 1536 (96*16) (từ 256/64 = 4x4)

        # Bottleneck: Lấy từ stage4 (kích thước 4x4)
        self.bottleneck = ConvBlock(config.embed_dim * 16, config.embed_dim * 8) # Từ 1536 kênh về 768 kênh

        # Các Decoder Block
        # Nối output của bottleneck (768 kênh) với stage3 (768 kênh)
        self.decoder4 = DecoderBlock(in_channels=config.embed_dim * 8, skip_channels=config.embed_dim * 8, out_channels=config.embed_dim * 4) # Output 384 kênh
        # Nối output của decoder4 (384 kênh) với stage2 (384 kênh)
        self.decoder3 = DecoderBlock(in_channels=config.embed_dim * 4, skip_channels=config.embed_dim * 4, out_channels=config.embed_dim * 2) # Output 192 kênh
        # Nối output của decoder3 (192 kênh) với stage1 (192 kênh)
        self.decoder2 = DecoderBlock(in_channels=config.embed_dim * 2, skip_channels=config.embed_dim * 2, out_channels=config.embed_dim * 1) # Output 96 kênh
        # Nối output của decoder2 (96 kênh) với patch_embeddings (96 kênh)
        self.decoder1 = DecoderBlock(in_channels=config.embed_dim * 1, skip_channels=config.embed_dim, out_channels=config.embed_dim // 2) # Output 48 kênh

        # Final Upsample để đạt kích thước ban đầu (nếu cần)
        # Từ 48 kênh lên 24 kênh, kích thước 128x128 -> 256x256
        self.final_upsample = DecoderBlock(in_channels=config.embed_dim // 2, skip_channels=0, out_channels=config.embed_dim // 4) 

        self.final_conv = nn.Conv2d(config.embed_dim // 4, num_classes, kernel_size=1)
        # self.sigmoid = nn.Sigmoid() # LOẠI BỎ SIGMOID Ở ĐÂY, BCEWithLogitsLoss sẽ xử lý!

    def forward(self, x):
        outputs = self.swin(pixel_values=x, output_hidden_states=True)
        
        # Lấy các hidden_states tương ứng và chuyển đổi sang định dạng (B, C, H, W)
        # SwinModel trả về hidden_states theo thứ tự: patch_embeddings, stage1, stage2, stage3, stage4
        
        # Hàm trợ giúp để chuyển đổi từ (batch_size, num_patches, embed_dim) sang (batch_size, embed_dim, H, W)
        def reshape_to_cnn_format(hs_tensor, embed_dim_val):
            batch_size, num_patches, _ = hs_tensor.shape
            side = int(np.sqrt(num_patches))
            return hs_tensor.permute(0, 2, 1).reshape(batch_size, embed_dim_val, side, side)

        # Lấy các feature maps từ encoder
        f_patch_embed = reshape_to_cnn_format(outputs.hidden_states[0], self.swin.config.embed_dim) # 96 kênh, 64x64
        f_stage1 = reshape_to_cnn_format(outputs.hidden_states[1], self.swin.config.embed_dim * 2)  # 192 kênh, 32x32
        f_stage2 = reshape_to_cnn_format(outputs.hidden_states[2], self.swin.config.embed_dim * 4)  # 384 kênh, 16x16
        f_stage3 = reshape_to_cnn_format(outputs.hidden_states[3], self.swin.config.embed_dim * 8)  # 768 kênh, 8x8
        f_stage4 = reshape_to_cnn_format(outputs.hidden_states[4], self.swin.config.embed_dim * 16) # 1536 kênh, 4x4 (đây là đầu vào cho bottleneck)

        # Bottleneck
        x = self.bottleneck(f_stage4) # (B, 1536, 4, 4) -> (B, 768, 4, 4)

        # Decoder
        x = self.decoder4(x, f_stage3) # (B, 768, 4, 4) + (B, 768, 8, 8) -> (B, 384, 8, 8)
        x = self.decoder3(x, f_stage2) # (B, 384, 8, 8) + (B, 384, 16, 16) -> (B, 192, 16, 16)
        x = self.decoder2(x, f_stage1) # (B, 192, 16, 16) + (B, 192, 32, 32) -> (B, 96, 32, 32)
        x = self.decoder1(x, f_patch_embed) # (B, 96, 32, 32) + (B, 96, 64, 64) -> (B, 48, 64, 64)
        
        x = self.final_upsample(x) # (B, 48, 64, 64) -> (B, 24, 128, 128)

        # Chuyển đổi về kích thước IMG_SIZE (256x256) nếu cần thiết ở bước cuối cùng
        if x.shape[2:] != (self.IMG_SIZE, self.IMG_SIZE):
            x = nn.functional.interpolate(x, size=(self.IMG_SIZE, self.IMG_SIZE), mode='bilinear', align_corners=True)

        outputs = self.final_conv(x) # (B, 24, 256, 256) -> (B, 1, 256, 256)
        # outputs = self.sigmoid(outputs) # LOẠI BỎ Ở ĐÂY!

        return outputs

In [10]:
def calculate_metrics(predicted_masks, true_masks, smooth=1e-6):
    intersection = (predicted_masks * true_masks).sum()
    union = (predicted_masks + true_masks).sum() - intersection
    
    iou = (intersection + smooth) / (union + smooth)
    
    dice = (2. * intersection + smooth) / ((predicted_masks.sum() + true_masks.sum()) + smooth)
    f1_score = dice 

    return iou.item(), f1_score.item()

# Thêm tham số start_epoch và best_val_loss_so_far để tiếp tục từ checkpoint
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, callbacks_config, start_epoch=0, best_val_loss_so_far=float('inf')):
    best_val_loss = best_val_loss_so_far 
    patience_counter = 0
    model_checkpoint_path = callbacks_config.get('checkpoint_path', 'swin_unet_best_pytorch.pth')

    for epoch in range(start_epoch, num_epochs):
        model.train()
        running_loss = 0.0
        running_iou = 0.0
        running_f1 = 0.0
        print(f"Epoch {epoch+1}/{num_epochs} Bắt đầu...")
        for batch_idx, (images, masks) in enumerate(train_loader):
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, masks) # BCEWithLogitsLoss nhận logits và targets

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

            # Để tính toán metrics, chúng ta cần chuyển logits sang xác suất rồi sang mask nhị phân
            predicted_masks = (torch.sigmoid(outputs) > 0.5).float() 
            
            batch_iou, batch_f1 = calculate_metrics(predicted_masks, masks)
            running_iou += batch_iou * images.size(0)
            running_f1 += batch_f1 * images.size(0)

            if (batch_idx + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_iou = running_iou / len(train_loader.dataset)
        epoch_f1 = running_f1 / len(train_loader.dataset)
        print(f"Epoch {epoch+1} Kết thúc - Mất mát Huấn luyện: {epoch_loss:.4f}, IoU Huấn luyện: {epoch_iou:.4f}, F1-Score Huấn luyện: {epoch_f1:.4f}")

        model.eval()
        val_loss = 0.0
        val_iou = 0.0
        val_f1 = 0.0

        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(DEVICE)
                masks = masks.to(DEVICE)

                outputs = model(images)
                loss = criterion(outputs, masks)

                val_loss += loss.item() * images.size(0)

                predicted_masks = (torch.sigmoid(outputs) > 0.5).float() 
                
                batch_iou, batch_f1 = calculate_metrics(predicted_masks, masks)
                val_iou += batch_iou * images.size(0)
                val_f1 += batch_f1 * images.size(0)

        val_loss /= len(val_loader.dataset)
        val_iou /= len(val_loader.dataset)
        val_f1 /= len(val_loader.dataset)
        print(f"Mất mát Xác thực: {val_loss:.4f}, IoU Xác thực: {val_iou:.4f}, F1-Score Xác thực: {val_f1:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            print(f"Mất mát xác thực tốt nhất được cập nhật: {best_val_loss:.4f}. Lưu mô hình và trạng thái...")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_loss': best_val_loss,
            }, model_checkpoint_path)
        else:
            patience_counter += 1
            print(f"Mất mát xác thực không cải thiện. Sự kiên nhẫn: {patience_counter}/{callbacks_config['patience']}")
            if patience_counter >= callbacks_config['patience']:
                print("Dừng sớm!")
                break

# --- Khởi tạo Dataset và DataLoader ---
train_dataset = CrackDetectionDataset(train_img_paths, train_mask_paths, transform=train_transform)
val_dataset = CrackDetectionDataset(val_img_paths, val_mask_paths, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0) # num_workers > 0 nếu bạn muốn tải dữ liệu song song
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# --- Khởi tạo mô hình, optimizer, criterion ---
model = SwinUNet(input_channels=3, num_classes=1).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss() # Đây là hàm loss đúng khi đầu ra là logits

# In cấu trúc mô hình để kiểm tra
print(model)

callbacks_config = {
    'patience': 30, # Số epoch chờ trước khi dừng sớm nếu val_loss không cải thiện
    'checkpoint_path': 'swin_unet_best_pytorch.pth'
}

# --- Logic để tiếp tục huấn luyện từ checkpoint ---
start_epoch = 0
best_val_loss_so_far = float('inf')
checkpoint_path = callbacks_config['checkpoint_path']

if os.path.exists(checkpoint_path):
    print(f"Phát hiện checkpoint tại {checkpoint_path}. Đang tải để tiếp tục huấn luyện...")
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1 
    best_val_loss_so_far = checkpoint['best_val_loss']
    
    print(f"Đã tải checkpoint từ Epoch {start_epoch-1}. Tiếp tục huấn luyện từ Epoch {start_epoch}.")
    print(f"Mất mát xác thực tốt nhất trước đó: {best_val_loss_so_far:.4f}")
else:
    print("Không tìm thấy checkpoint. Bắt đầu huấn luyện từ đầu (Epoch 0).")

print("\nBắt đầu huấn luyện mô hình Swin-Unet...")
train_model(model, train_loader, val_loader, optimizer, criterion, 
            num_epochs=10000, callbacks_config=callbacks_config,
            start_epoch=start_epoch, best_val_loss_so_far=best_val_loss_so_far)

print("\nQuá trình huấn luyện đã hoàn tất.")

ValueError: out_features must be a subset of stage_names: ['stem', 'stage1', 'stage2', 'stage3', 'stage4'] got ['patch_embeddings', 'stage1', 'stage2', 'stage3', 'stage4']